In [7]:
import os
import copy
import json
import numpy as np
import pandas as pd
from ultralytics import YOLO

def add_f1_scores(metrics: dict) -> dict:
    """Calculate and add F1 scores to a copy of the given metrics dictionary, keeping the desired order."""
    new_metrics = copy.deepcopy(metrics)  # Create a copy to avoid modifying the original

    # Extract precision and recall for (B)
    precision_B = new_metrics.get('metrics/precision(B)', 0)
    recall_B = new_metrics.get('metrics/recall(B)', 0)
    F1_B = 2 * (precision_B * recall_B) / (precision_B + recall_B) if (precision_B + recall_B) > 0 else 0

    # Extract precision and recall for (M)
    precision_M = new_metrics.get('metrics/precision(M)', 0)
    recall_M = new_metrics.get('metrics/recall(M)', 0)
    F1_M = 2 * (precision_M * recall_M) / (precision_M + recall_M) if (precision_M + recall_M) > 0 else 0

    # Create a new dictionary with the desired order
    ordered_metrics = {}
    for key, value in new_metrics.items():
        ordered_metrics[key] = value
        if key == 'metrics/recall(B)':
            ordered_metrics['metrics/F1_score(B)'] = np.float64(F1_B)
        if key == 'metrics/recall(M)':
            ordered_metrics['metrics/F1_score(M)'] = np.float64(F1_M)

    return ordered_metrics


def safe_validate(model_path, extra_config):
    model = YOLO(model_path, task="segment")
    val_metrics = model.val(**extra_config)
    return val_metrics

def validate_experiment(dataframe, parameters):
    model_pt_path = parameters["model_pt_path"]
    validation_params = parameters["validation_params"]
    model_name = parameters["model_name"]
    dataset_name = parameters["dataset_name"]
    optimizer = parameters["optimizer"]
    model_format = parameters["format"]
    
    val_results = safe_validate(model_pt_path, validation_params)
    data = add_f1_scores(val_results.results_dict) | val_results.speed
    cleaned_data = {key.replace("metrics/", ""): value for key, value in data.items()}
    cleaned_data.update(Model=model_name, Dataset=dataset_name, Optimizer=optimizer, Format=model_format)
    row_data = pd.DataFrame([cleaned_data])
    dataframe = pd.concat([dataframe, row_data], ignore_index=True)


def validate_run(config_file, results_path):
    with open(config_file, "r") as f:
        config_dict = json.load(f)
        
    dataframe = pd.DataFrame()

    for key, config in config_dict.items():
        dataset_name, temp = os.path.split(key)
        model_name, optimizer = temp.split("_")

        hyperparams = config["hyperparams"]
        done = config["done"]
        trt32 = config["trt32"]
        trt16 = config["trt16"]
        trt8 = config["trt8"]

        # Paths
        dataset_yaml = hyperparams["data"]
        model_pt_path = os.path.join(hyperparams["project"],hyperparams["name"], "weights", "best.pt")
        model_trt32_path = os.path.join(hyperparams["project"],hyperparams["name"], "weights", "best_trt_fp32.engine")
        model_trt16_path = os.path.join(hyperparams["project"],hyperparams["name"], "weights", "best_trt_fp16.engine")
        model_trt8_path = os.path.join(hyperparams["project"],hyperparams["name"], "weights", "best_trt_int8.engine")

        validation_params = {"data": dataset_yaml, "device": "cuda:0", "split": "val"}

        if done:
            parameters = {"model_pt_path": model_pt_path, "validation_params": validation_params, "model_name": model_name,
                          "dataset_name": dataset_name, "optimizer": optimizer, "model_format": "Pytorch"}
            validate_experiment(dataframe, parameters)
        if trt32:
            parameters = {"model_pt_path": model_trt32_path, "validation_params": validation_params, "model_name": model_name,
                          "dataset_name": dataset_name, "optimizer": optimizer, "model_format": "TensorRT-F32"}
            validate_experiment(dataframe, parameters)
        if trt16:
            parameters = {"model_pt_path": model_trt16_path, "validation_params": validation_params, "model_name": model_name,
                          "dataset_name": dataset_name, "optimizer": optimizer, "model_format": "TensorRT-F16"}
            validate_experiment(dataframe, parameters)
        if trt8:
            parameters = {"model_pt_path": model_trt8_path, "validation_params": validation_params, "model_name": model_name,
                          "dataset_name": dataset_name, "optimizer": optimizer, "model_format": "TensorRT-INT8"}
            validate_experiment(dataframe, parameters)

    dataframe.to_csv(results_path, index=False)


In [3]:
config_file = "training/run1.json"

with open(config_file, "r") as f:
    config_dict = json.load(f)

dataframe = pd.DataFrame()

key = "Deepfish_LO/yolov8n-seg_SGD"
dataset, temp = os.path.split(key)
modelo, optimizador = temp.split("_")

config = config_dict[key]
done = config["done"]
trt32 = config["trt32"]
trt16 = config["trt16"]
trt8 = config["trt8"]
hyperparams = config["hyperparams"]

# Paths
dataset_yaml = hyperparams["data"]
model_pt_path = os.path.join(hyperparams["project"],hyperparams["name"], "weights", "best.pt")
model_trt32_path = os.path.join(hyperparams["project"],hyperparams["name"], "weights", "best_trt_fp32.engine")
model_trt16_path = os.path.join(hyperparams["project"],hyperparams["name"], "weights", "best_trt_fp16.engine")
model_trt8_path = os.path.join(hyperparams["project"],hyperparams["name"], "weights", "best_trt_int8.engine")

validation_params = {"data": dataset_yaml, "device": "cuda:0", "split": "val", "batch": 8}

if done:
    val_results = safe_validate(model_pt_path, validation_params)
    data = val_results.results_dict | val_results.speed
    cleaned_data = {key.replace("metrics/", ""): value for key, value in data.items()}
    cleaned_data.update(Model=modelo, Dataset=dataset, Optimizer=optimizador, Format="Pytorch")
    row_data = pd.DataFrame([cleaned_data])
    dataframe = pd.concat([dataframe, row_data], ignore_index=True)
if trt32:
    val_results = safe_validate(model_trt32_path, validation_params)
    data = val_results.results_dict | val_results.speed
    cleaned_data = {key.replace("metrics/", ""): value for key, value in data.items()}
    cleaned_data.update(Model=modelo, Dataset=dataset, Optimizer=optimizador, Format="TensorRT-F32")
    row_data = pd.DataFrame([cleaned_data])
    dataframe = pd.concat([dataframe, row_data], ignore_index=True)
if trt16:
    val_results = safe_validate(model_trt16_path, validation_params)
    data = val_results.results_dict | val_results.speed
    cleaned_data = {key.replace("metrics/", ""): value for key, value in data.items()}
    cleaned_data.update(Model=modelo, Dataset=dataset, Optimizer=optimizador, Format="TensorRT-F16")
    row_data = pd.DataFrame([cleaned_data])
    dataframe = pd.concat([dataframe, row_data], ignore_index=True)
if trt8:
    val_results = safe_validate(model_trt8_path, validation_params)
    data = val_results.results_dict | val_results.speed
    cleaned_data = {key.replace("metrics/", ""): value for key, value in data.items()}
    cleaned_data.update(Model=modelo, Dataset=dataset, Optimizer=optimizador, Format="TensorRT-INT8")
    row_data = pd.DataFrame([cleaned_data])
    dataframe = pd.concat([dataframe, row_data], ignore_index=True)    


Ultralytics 8.3.75 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 11912MiB)


YOLOv8n-seg summary (fused): 195 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs


val: Scanning /home/memorista01/tesis/datasets_yolo/Deepfish_LO/labels/valid.cache... 66 images, 59 backgrounds, 0 corrupt: 100%|██████████| 125/125 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:01<00:00, 14.88it/s]


                   all        125         79      0.883      0.953      0.966      0.806      0.925      0.933      0.971      0.761
Speed: 0.4ms preprocess, 2.6ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/segment/val6
